SEGMENTING AND CLUSTERING - FIRST QUESTION
=========================

In [8]:
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#Create Dataframe
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df
#Check shape prior to dropping Not Assigned
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)  

The dataframe has 12 boroughs and 287 neighborhoods.


__TRANSFORMING DATAFRAME__

In [9]:
#Drop "Not Assigned" Boroughs
df=df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

#Check the new shape
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
        )
)  

The dataframe has 11 boroughs and 210 neighborhoods.


In [10]:
#Replacing Not Assigned Neighborhoods with Burough Name.
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']

#Grouping by Postcode
df_grouped = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_grouped.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


__PRINTING HOW MANY ROWS THE DATAFRAME HAS__

In [11]:
#How many rows the dataframe has.
print('The dataframe has {} rows.'.format(
        df_grouped.shape[0]
        )
)  

The dataframe has 103 rows.


SEGMENTING AND CLUSTERING - SECOND QUESTION
=========================

__Loading CSV-File__

In [15]:
#Getting the CSV-file
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


__Merging the dataframe with coordinates__

In [17]:
#Rename the column to match df_grouped
coordinates.rename(columns={"Postal Code": "Postcode"}, inplace=True)

In [18]:
#Create new dataframe with combined data from both previous.
df=df_grouped.merge(coordinates, on="Postcode", how="left")
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
